# My First XGBoost Algorithm
### With One Hot Encoding, Pipelines and Cross Validation

Hi! I'm Douglas and I'm a Data Science enthusiast trying to learn everything I think I need to become a Data Scientist.

I took another step outside my studying plans and did one more quick Kaggle course on [**Intermediate Machine Learning**](https://www.kaggle.com/learn/intermediate-machine-learning). There they use the famous housing prices example with both Melbourn and Iowa data and extend their [Introductory course](https://medium.com/r/?url=https%3A%2F%2Fwww.kaggle.com%2Flearn%2Fintro-to-machine-learning) with more intermediate concepts like dealing with missing values and categorical features, pipelines, cross-validation and, most important of all (for me, who hadn't seen it before) the XGBoost algorithm. 

![Kaggle Completion Certificate](img/certificate_2.png)

If you read my tutorialish article on the content of their first course [**My First Machine Learning Algorithm**](https://medium.com/@douglas.rochedo/decision-tree-and-random-forest-for-predicting-used-car-prices-9233478818b0) you will notice that I, unlike Kaggle, used a Used Cars dataset to exemplify and showcase these concepts. Let's take a look into it.

### The Data

In [1]:
import pandas as pd

dataset = pd.read_csv("car data.csv")
print(dataset.shape)
dataset.head()

(301, 9)


,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


Here we have 301 used cars and 9 features (or atributes) of each one of them. The first thing we do when we first meet a new dataset is check for null values and duplicates. I have already done that in my above-mentioned article and found there were no null values but a couple duplicates. So we should merely drop those duplicated cars. 

In [58]:
dataset.drop_duplicates(inplace=True)
print(dataset.shape)

(299, 9)


Next, we should remove from our dataset the target feature, that is, the thing we want to predict with our algorithm, called the label. In this dataset, that would be the "Present_Price" column. 

In [56]:

y = dataset['Present_Price']
X = dataset.drop('Present_Price', axis=1)
print(X.shape, y.shape)

(299, 8) (299,)


Among the features in this dataset, some are continuous and some are categorical. Continuous features are allways numeric, but categorical features may be numbers, strings or even some other type of variable. In this specific case, all of our categorical features are strings, Machine Learning algorithms in general cannot handle categorical features (that is why we dumped these features in the last article). XGBoost is one of them, as is Random Forest, so we will have to turn those features into binary numerical data. To do that we will perform One Hot Encoding.

### One Hot Encoding

I can find no better way to explain this than what has been said in the Kaggle course this article is based on, so I will quote it:

*"One-hot encoding creates new columns indicating the presence (or absence) of each possible value in the original data."*

<img src="img/one_hot_encoding.png" alt="drawing" height="300"/>

In this example image also taken from Kaggle, we see that the Color feature can take one of 3 possible categorical values: Red, Green and Yellow, which are expressed in the form of strings. To turn these values into binary numerical ones, we create three new columns, one for each possible color, and have it be 1 if it is that color and 0 if it is not. This technique is really good when dealing with a small number of possible values (or cardinality) like in this example, but it performns very poorly when the cardinality is too high like, let's say, above 10-15 different possible values. 

To perform One Hot Encoding we will use Scikit-learn's *OneHotEncoder* class. That class shall fit to a Data Frame and perform One Hot Encoding in all of it's columns. Thus, we need to pass it only the columns we want the transfomation to be made on. 

In [60]:
cat_col_names = []
num_col_names = []

for col in X.columns:
    if X[col].dtype == 'object':
        cat_col_names.append(col)
    else:
        num_col_names.append(col)

print(cat_col_names)
print(num_col_names)

['Car_Name', 'Fuel_Type', 'Selling_type', 'Transmission']
['Year', 'Selling_Price', 'Driven_kms', 'Owner']


In Python, we can use the *dtype* attribute of a column to determine it's type and all strings are *object*. Thus, separating the column names by that measure gives us two lists: one with the names of the categorical features and one with the names of the continuous numerical ones. 

Now, to find the cardinality of each of those features, we will use Pandas' *unique()* function to determine what are the possibile values those feature can take and count them. 

In [61]:
for col in cat_col_names:
    print(col, len(X[col].unique()))

Car_Name 98
Fuel_Type 3
Selling_type 2
Transmission 2


Here we see that Fuel_Type, Selling_type and Transmisson have pretty small cardinality, they are perfect candidates for One Hot Encoding. On the other hand, we now see we have 98 diferent car names - or car models, if you'd rather - in our dataset. Is that impossible to One Hot Encode? No, it is not. But in this case, with 98 diferent models in a dataset of only around 300 cars, it will most definely only dampen our performance. So, our solution will be to remove the Car_Name column from our dataset completly with the following code.

In [62]:
cat_col_names = cat_col_names[1:]
print(cat_col_names)

['Fuel_Type', 'Selling_type', 'Transmission']


Further on, we must now actually split our dataset (not only the column names) between numerical and categorical.

In [63]:
cat_cols = X[cat_col_names]
num_cols = X[num_col_names]

Now, to the real sweet part, we import the *OneHotEncoder* from sklearn and use it to actually encode our categorical features. 

In [64]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)
cat_cols_OH = pd.DataFrame(encoder.fit_transform(cat_cols))
cat_cols_OH.head()


,0,1,2,3,4,5,6
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0


Well, I understand you are most likely very bothered by the lack of names on the columns. In reality, since we are doing this encoding for the sole purpose of feeding this data into an algorithm, it is nothing to worry about. Actually, later in this article we will do things in a way we can't even see the table created. But, just to show you how it would look, we can use the encoders method *get_feature_names_out* to have the names back. And, on top of it, the method also appends an explicative name on the end, just like so:

In [65]:
cat_cols_OH.index = cat_cols.index

OH_col_names = encoder.get_feature_names_out(cat_col_names)
cat_cols_OH.columns = OH_col_names
cat_cols_OH.head()

,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Selling_type_Dealer,Selling_type_Individual,Transmission_Automatic,Transmission_Manual
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0


Another small change we might care to make is to drop some ot these columns. 

Using the Fuel type as an example, note that if the car's fuel type is either Diesel or Petrol, it won't be CNG, right? (at least that is the case for this dataset) Also, if the car is neither Diesel nor Petrol, we can say for sure it is CNG, isn't it? That means we can easly drop the CNG column without loosing the information of wheter it is or not CNG. That is also the case for the two other columns. This is a good practice because it lowers the amount of columns we have and, thus, lowers the chance of Overfitting and the size of our dataset. In this case maybe that wouldn't make such a diference, but we will do it anyway. 

Lucky for ws, the *OneHotEncoder* himself does that for us if we simply tell it *drop='first'* to drop the first column of each variable it encodes.

In [66]:
encoder = OneHotEncoder(sparse = False, drop='first')
cat_cols_OH = pd.DataFrame(encoder.fit_transform(cat_cols))

cat_cols_OH.index = cat_cols.index

OH_col_names = encoder.get_feature_names_out(cat_col_names)
cat_cols_OH.columns = OH_col_names
cat_cols_OH.head()

,Fuel_Type_Diesel,Fuel_Type_Petrol,Selling_type_Individual,Transmission_Manual
0,0.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,1.0


At last, we only concatenate these encoded columns with the numerical columns we had at the beginning ans we have our dataset ready to be used

In [67]:
X_OH = pd.concat([num_cols, cat_cols_OH], axis=1)
X_OH.head()

,Year,Selling_Price,Driven_kms,Owner,Fuel_Type_Diesel,Fuel_Type_Petrol,Selling_type_Individual,Transmission_Manual
0,2014,3.35,27000,0,0.0,1.0,0.0,1.0
1,2013,4.75,43000,0,1.0,0.0,0.0,1.0
2,2017,7.25,6900,0,0.0,1.0,0.0,1.0
3,2011,2.85,5200,0,0.0,1.0,0.0,1.0
4,2014,4.60,42450,0,1.0,0.0,0.0,1.0


### XGBoost

Finally, the part everyone (or at least me) was expecting. **XGBoost**, or rather, **Extreme Gradient Boosting** is an Ensemble Machine Learning Alrotithm, which means that, instead of using a single model, it uses a group of many algorithms (or estimators, as itself calls them) and, in the end, gives the answer as if it was a single one. Unlike the Random Forest, which we've talked about in my other article, Gradient Boosting does not take the average of the precictions of each estimator, but rather uses the output of an estimator do "boost" the next using gradient descent. That (at least in theory), makes it a better algorithm than Random Forest.

XGBoost is an enhanced Gradient Boosting, or, as some like calling it, "Gradient Boosting on steroids" (that is why it is called Extreme Gradient Boosting). Although, these enchancements are only time-performance-wise. That means it shall not have better results than ordinary Gradient Boosting, but shall run much faster and thus, this is amongst the State-of-the-Art algorithms in Machine Learning.

To use it one must import the *xgboost* module and get the best suited class/estimator for the job to be done. In our case it's the *XGBRegressor*. This class works perfectly with Sciki-learn, so it is our best choice at the moment. Other then that, we follow up with the same precedures as always: split the data, fit the model, predict it and evaluate it with some error metric, in our case the Mean Absolute Error.

In [68]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error


X_train, X_valid, y_train, y_valid = train_test_split(X_OH, y, random_state=7)

model = XGBRegressor(random_state=7)
model.fit(X_train,y_train)
predictions = model.predict(X_valid)

mae_df = mean_absolute_error(predictions, y_valid)
print(mae_df)

1.2305056664148966


An there we have it! XGBoost applied to Used Cars Price predicion. We could as well stop ourselves there, but that quick course, even though quick, have some other interesting insights we should talk about here. 


### K-Fold Cross-validation

So far we have been using a simple train-test split to evaluate our model, but that is not the best way of doing it because it relies too much on randomness. If we make a good split we have good results, if we have a bad split we have bad ones. We can confirm that by trying diferent values for our random_state:

In [69]:
for rs in range(10):
    X_train, X_valid, y_train, y_valid = train_test_split(X_OH, y, random_state=rs)

    model = XGBRegressor(random_state=7)
    model.fit(X_train,y_train)
    predictions = model.predict(X_valid)

    mae_df = mean_absolute_error(predictions, y_valid)
    print(rs, mae_df)

0 1.3896726737149556
1 1.8431405510171253
2 1.1715617687384285
3 1.454248183631897
4 1.3375711557865142
5 1.5109027722962696
6 1.0504326105690003
7 1.2305056664148966
8 1.3480223269399008
9 0.9464802797190348


See? There is a number of possible splits and each gives a diferent result. The one we were using isn't even the best one. 

Does that mean we should use a for loop like this one and always look for the split that gives us the smallest error? Most definetly NO! This error is but an estimative of how well this model is going to work on new data, data that it hasn't seen before. If you just want a small error to show off to your Data Science friends,  you may do that, but the actual idea you should have when developing a model is to make it generalistic, to make it predict well on data it has never seen before.

And how do we achieve that? A really good option is evaluating our model with K-Fold Cross Validation instead. With that technique, instead of spliting the data once with the default proportions (80% for training and 20% for validation), it does it K times, each time with a diferent split. In the end, we have K diferent error-values to evaluate and, thus, have a far better understanding of our model. 

In order to do that with Scikit-learn, we can use *cross_val_score*. You feed it the model you want to fit, the dataset (both features and label), the scoring method - or the error metric, among those available - and the value of K, here called *cv*. The cross_val_score was built to work, among other error metrics, with the R2 Score, which is bigger the better the model is. Thus, it seeks to always increase the score, insetead of decreasing it. That is why we use negative MAE instead of just MAE and why we have a minus sign in front of the whole function. 

In [70]:
from sklearn.model_selection import cross_val_score

cv_scores = -cross_val_score(model, X_OH, y, scoring='neg_mean_absolute_error', cv=10)
print(cv_scores)

mae_cv = cv_scores.mean()
print(mae_cv)

[1.08958018 1.64400977 5.10839024 1.03694723 0.16596069 0.1813305
 1.01518167 1.35696128 1.06405625 1.12627295]
1.378869075274707


And there we have it. The most common values for K (or CV) are 5 and 10, so we used latter. You can clearly see we have some divergence in the results of each run, so the average of the five gives the best possible idea of how this model would perform with external, unknown and never seen data. 

*"But the result was worse than before!"*

Yes, it was. This technique is the first one I've talked about here that does not seek to increse our performance, but rather to gives a better understanding of how our model is performing. That may not be the best goal when, as I've said, you just want to have the best numbers, but is most definetly the best course of action if you are going to put your model to production. 

### Pipeline

Here is one more technique we can and should use when developing Machine Learning models, specially when focused on production. Here we only did a small number of encodings and transformations on the dataset, but more often than not it requires far more transformations, data cleaning and wrangling than what we have here. If we do all of these step by step on each and every iteration, it can get really messy. To help us have cleaner code and facilitate the iterations of our coding, we have **Pipelines**. 

The *ColumnTransformer* class isn't quite a pipeline, but it suits a similar role. With this module we can set a transformer or a number of transformers (in the form of a pipeline) for a specific set of columns. Here, for example, we can use to perform One Hot Encoding only on the categorical columns whose names are in the *cat_col_names* list. It also has the possibility to *passthrough*, that is, to perform no transformation on that set of columns and to *drop* a set of columns (in this case the Car_Name column we droped in the beginning). The created class can be used to fit and transform data as if it was a single transformer class.

The actual *Pipeline* recieves the *steps* argument, a list of transformers and an estimator at the end. These transformers, unlike in the Column Transformer, do not apply one in each set of columns, but all of them apply on the dataset in the order they are listed. In this case, first the whole dataset goes through the Column Transformer and then it gets to the model. The Pipeline works as an estimator and can be used to fit, predict, and every other method an estimator can use. 

At last, we can also feed this estimator to the cross_val_score function and look at its results.



In [72]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


preprocessor = ColumnTransformer([
    ("num_cols", 'passthrough', num_col_names),
    ("cat_cols", OneHotEncoder(sparse=False, handle_unknown= 'ignore', drop='first'), cat_col_names)
], remainder='drop')

model = XGBRegressor(random_state=7)

pipe = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', model)
])


cv_scores = -cross_val_score(pipe, X, y, scoring='neg_mean_absolute_error', cv=10)
mae = cv_scores.mean()

print(cv_scores)
print(mae)


[1.08958018 1.64400977 5.10839024 1.03694723 0.16596069 0.1813305
 1.01518167 1.35696128 1.06405625 1.12627295]
1.378869075274707


We can see the results and they are exactly the same as before, prooving this to be a much better and cleaner way of organizing our models code. 

In any case, this is my quick tutorial/walkthrough on how to create your first Extreme Gradiente Boosting Machine Learning algorithm to predict the price of used cars with One Hot Encoding, Cross Validation and Pipelines. The notebook file will be available for download on my GitHub [**here**](https://github.com/DouglasRFLeite/UsedCarsPricePrediction.git) alongside the dataset used. I hope this helped you in any way but really, as I always say, even if it didn’t help you, it helped me tremendously so it’s worth it. See you soon!